In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def show_image(image):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    fig = plt.figure(frameon=False)
    plt.imshow(image_rgb)
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    

In [ ]:
def solution(image_path_a, image_path_b):
    ############################
    ############################
    ## image_path_a is path to the non-flash high ISO image
    ## image_path_b is path to the flash low ISO image
    ############################
    ############################
    ## comment the line below before submitting else your code wont be executed##
    # pass
    image = cv2.imread(image_path_b)
    return image


In [ ]:
show_image(solution('ultimate_test/1_a.jpg', 'ultimate_test/1_b.jpg'))

In [ ]:
show_image(solution('ultimate_test/2_a.jpg', 'ultimate_test/2_b.jpg'))